In [1]:
import pandas as pd
import numpy as np


In [2]:
pd.__version__

'1.4.2'

In [3]:
def check_qc(_series):
    barcode = _series['SampleBarcode']
    anti_A = _series['Anti-A']
    result_text = _series['ResultText']
    dat = _series['DAT poly']
    mono = c3d = _series['C3d']
    igg = _series['IgG']
    screen = _series['I']
    
    # qc1
    if barcode.startswith("0871"):
            if anti_A!="":
                return result_text == "A Rh D negative"
            
            if screen !="":
                return result_text == "ABS positive"
            
            if dat !="":
                return result_text == "DAT negative"
    
    # qc2
    if barcode.startswith("0872"):
            if anti_A!="":
                return result_text == "B Rh D positive"
            
            if screen !="":
                return result_text == "ABS positive"
            
            if dat !="":
                return result_text == "DAT negative"
    
    # qc7
    if barcode.startswith("0877"):           
            if dat !="":
                return result_text == "DAT positive"
            
            if mono !="":
                return (result_text == "DAT positive") and (c3d == "-")
            
            if screen !="":
                return result_text == "ABS negative"
            
    # qc8
    if barcode.startswith("0878"):           
            if dat !="":
                return result_text == "DAT positive"
            
            if mono !="":
                return (result_text == "DAT positive") and (igg == "-")
            
            if screen !="":
                return result_text == "ABS negative"
    
    # lab qc1
    if barcode.startswith("0841"):
        if screen !="":
                return result_text == "ABS negative"
    
    # lab qc2
    if barcode.startswith("0842"):
        if screen !="":
                return result_text == "ABS negative"

            
    
    print(_series)
    return False

In [4]:
def check_liq(series):
    
    for c in series.index:
        if series[c] == 'LIQ':
            return True
        
    return False

In [5]:
try:
    df = pd.read_xml("p:/PrintOut_StatReportForExport.xml", xpath="//IHComTable")
except:
    df = pd.read_xml("PrintOut_StatReportForExport.xml", xpath="//IHComTable")

    
now_string = pd.Timestamp.now().strftime('%Y%m%d_%H%M')

try:
    df.to_excel(f"F:/Printer/ALL_{now_string}.xlsx")
except OSError:
    df.to_excel(f"ALL_{now_string}.xlsx")

In [6]:
# qc only
qc_df = df[df['SampleBarcode'].str.contains("QC")]
qc_df = qc_df.copy()
qc_df['BGText'] = qc_df['BGText'].fillna("")
qc_df['BGTextABScrDAT'] = qc_df['BGTextABScrDAT'].fillna("")
# remove not interpretable
qc_df = qc_df[~(qc_df['BGText'].str.contains("not"))]
qc_df = qc_df[~(qc_df['BGTextABScrDAT'].str.contains("not"))]

In [7]:
# reform table as TestDate, SampleBarcode, VerifiedByUser, ResultText, [tests....]

group = qc_df.groupby(['SampleBarcode','TestDate'])

sample_df_list=[]

for key in group.groups.keys():
    sample_df = group.get_group(key).set_index("WellName")
    sample_result_series = sample_df.FinalResultText
    
    sample_df['ResultText'] = sample_df['BGText']+sample_df['BGTextABScrDAT']
    sample_info_series = sample_df.iloc[0][['TestDate','SampleBarcode','VerifiedByUser', 'ResultText']]

    sample_df_list.append(pd.DataFrame(pd.concat([sample_info_series, sample_result_series])).T)
    

In [8]:
result_df = pd.concat(sample_df_list)

# remove unverified data
result_df=result_df[~result_df['VerifiedByUser'].isna()]

# fill na with ""
result_df.fillna("", inplace=True)

# remove DP
result_df = result_df.replace("DP", np.nan).dropna()

# remove LIQ
result_df = result_df.replace("LIQ", np.nan).dropna()


result_df.set_index("TestDate", inplace=True)

result_df.sort_index(inplace=True)
result_df.reset_index(inplace=True)

In [9]:


try:
    result_df.to_excel(f"F:/Printer/QC_{now_string}.xlsx")
except OSError:
    result_df.to_excel(f"QC_{now_string}.xlsx")
    

In [10]:
# print(result_df)

In [11]:
start_date = result_df['TestDate'].apply(pd.to_datetime).min().strftime('%Y/%m/%d')
end_date  = result_df['TestDate'].apply(pd.to_datetime).max().strftime('%Y/%m/%d')

title = f"{start_date}-{end_date}"

In [12]:
result_df['QC_check'] = result_df.apply(check_qc, axis=1)



In [13]:
print("QC failed:")
print(result_df[~result_df['QC_check']])

# result_df['QC_check'] = result_df['QC_check'].apply(lambda x: "Failed" if not x else "")
#remove result if qc failed
result_df = result_df[result_df['QC_check']].drop("QC_check", axis=1)

QC failed:
Empty DataFrame
Columns: [TestDate, SampleBarcode, VerifiedByUser, ResultText, I, II, III, Anti-A, Anti-B, Anti-D VI-, Ctl, A1, B, DAT poly, IgG, C3d, QC_check]
Index: []


In [14]:
dc_screening_df = result_df[result_df['IgG']!=""][["TestDate","SampleBarcode","VerifiedByUser","ResultText","Ctl", "IgG", "C3d"]]
dc_screening_df

,TestDate,SampleBarcode,VerifiedByUser,ResultText,Ctl,IgG,C3d
38,2022-08-05T13:00:20.677402+08:00,08770801220822QC,automatically accepted,DAT positive,-,+++,-
39,2022-08-05T13:00:20.677402+08:00,08780801220822QC,automatically accepted,DAT positive,-,-,+++
70,2022-08-08T16:16:06.166+08:00,08770801220822QC,949892,DAT positive,-,++,-
71,2022-08-08T16:16:06.344+08:00,08780801220822QC,949892,DAT positive,-,-,++
88,2022-08-10T13:58:00.203743+08:00,08770801220822QC,automatically accepted,DAT positive,-,+++,-
89,2022-08-10T14:00:59.20717+08:00,08780801220822QC,automatically accepted,DAT positive,-,-,+++
140,2022-08-15T11:05:10.686+08:00,08780801220822QC,949892,DAT positive,-,-,++
141,2022-08-15T11:05:10.899+08:00,08770801220822QC,949892,DAT positive,-,+++,-
180,2022-08-18T15:30:17.986+08:00,08770801220822QC,047916,DAT positive,-,++,-
181,2022-08-18T15:30:18.139+08:00,08780801220822QC,047916,DAT positive,-,-,++


In [15]:
result_html = result_df.to_html(index=False)
dc_html = dc_screening_df.to_html(index=False)

In [16]:
# calculate liq rate
df['OrgResultText'] = df['FinalResultText']
df.loc[~df['InstrumentResultText'].isnull(),['OrgResultText']] = df[~df['InstrumentResultText'].isnull()]['InstrumentResultText']

liq_rate = df['OrgResultText'].value_counts()['LIQ'] / df['OrgResultText'].size *100

print(f"LIQ RATE: {liq_rate:.2f} %")

LIQ RATE: 6.02 %


In [17]:
# LIQ rate per day
df['Date2'] = df['TestDate'].apply(pd.to_datetime).apply(lambda x: x.strftime("%Y%m%d"))

liq_count_df = pd.DataFrame(df[['Date2', 'OrgResultText']].value_counts().rename("LIQ")).reset_index()

liq_count_df = liq_count_df[liq_count_df['OrgResultText']=="LIQ"].sort_values("Date2").drop('OrgResultText', axis=1).set_index("Date2")

total_count_df = pd.DataFrame(df['Date2'].value_counts().rename("total")).sort_index()

liq_df = pd.merge(liq_count_df, total_count_df, how="outer", left_index=True, right_index=True).fillna(0)
liq_df['rate'] = liq_df['LIQ']/liq_df['total'] 


In [18]:
try:
    liq_df.to_excel(f"F:/Printer/LIQ_{now_string}.xlsx")
except OSError:
    liq_df.to_excel(f"LIQ_{now_string}.xlsx")

In [19]:
group = df.groupby(['SampleBarcode','TestDate'])

sample_df_list=[]

for key in group.groups.keys():
    sample_df = group.get_group(key).set_index("WellName")
    sample_result_series = sample_df.OrgResultText
    
    sample_df['ResultText'] = sample_df['BGText']+sample_df['BGTextABScrDAT']
    sample_info_series = sample_df.iloc[0][['TestDate','SampleBarcode']]

    sample_df_list.append(pd.DataFrame(pd.concat([sample_info_series, sample_result_series])).T)
    
result_df = pd.concat(sample_df_list)

result_df['LIQ'] = result_df.apply(check_liq, axis=1)

liq_rate_sample = result_df['LIQ'].value_counts()[True] / result_df['LIQ'].size *100

In [20]:
print(f"LIQ result rate: {liq_rate_sample:.2f} %")

LIQ result rate: 19.64 %


In [21]:
html_string = '''

<!DOCTYPE html>
<html>
<head>
<title>{time_range}</title>

{style}


</head>
<body>

<h1>成大醫院 病理部 血庫組 {title} 品管報表</h1>

<p>
<table width="100%">
	<thead>
		<tr style="text-align:right; font-size:12px; font-family:DFKai-sb;">
			<td>組長:_______________________</td>
		</tr>
	</thead>
	<tbody>
    
        <tr style="font-size:3px; font-family:calibri;">
            <td>{LIQ_rate}</td>
        </tr>
        
        
		<tr>
			<td> 
                
                {result}

            </td>
		</tr>
        
        
	</tbody>
</table>
</p>




</body>
</html>

'''

# print footer
'''
	@media print {
	  footer {
		position: fixed;
		bottom: -15px;
		right: 30px;
		width:100%
		border: 1px solid red;
		font-size: 15px;
		font-family: DFKai-sb;
	  }
       .content-block, p {
		page-break-inside: avoid;
	  }
	}
    
'''
        
style_string='''
<style>
    @media print {
      h1 {page-break-before: always;}
    }
    
    @media screen{
        body {
            width:90%;
            margin: auto auto;
        }
    }

    
	table, th, td{
		border: 0;
	}
    
    h1 {
		text-align:center;
		font-family:DFKai-sb;
		font-size: 20px;
	}
	
	.dataframe {
		width: 100%;
		font-family: calibri;
		font-size: 10px;
		border-bottom: 2px solid black;
		border-top: 2px solid black;
	}
	
	.dataframe th{
		border-bottom: 1px solid black;
		text-align: center;
	}
	
	.dataframe td:nth-child(-n+4){
		text-align: left;
	}
	
	.dataframe td{
		text-align: center;
	}
	
	.dataframe tr:nth-child(even){
		background-color: #f2f2f2;
	}
	
</style>
'''

In [22]:
with open("ih500.html", "w") as f:
     f.write(html_string.format(result=result_html,
                               style=style_string,
                               time_range=title,
                               title="IH-500",
                               LIQ_rate =f"LIQ rate= {liq_rate:.2f} %"))
        
with open("dc_screening.html", "w") as f:
     f.write(html_string.format(result=dc_html,
                               style=style_string,
                               time_range=title,
                               title="DC Screening-II",
                               LIQ_rate=""))
    
